In [91]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.ops import unary_union
from shapely.geometry import Point, MultiPolygon, Polygon, LineString
import topojson as tp
import h3
import os
pd.set_option('display.max_columns', None)

In [92]:
def merge_time_columns(time_df):
    time_df['row_col'] = time_df.apply(lambda row: [row['row'], row['col']], axis=1)
    time_df['x_y'] = time_df.apply(lambda row: [row['x'], row['y']], axis=1)
    time_df['x_ras_y_ras'] = time_df.apply(lambda row: [row['x_ras'], row['y_ras']], axis=1)
    time_df.drop(['row', 'col', 'x', 'y', 'x_ras', 'y_ras'], axis=1, inplace=True)
    return time_df

def join_points_poly(points_df, poly_df, join_rules):
    merged = poly_df
    joined_df = gpd.sjoin(points_df, poly_df)
    grouped = joined_df.groupby('index_right').agg(join_rules)
    merged = merged.merge(grouped, left_index=True, right_index=True)
    return merged

In [93]:
# Read in destination files as geodataframes
path = './unsynced-data/rwanda/travel-time'

# list the files in the path directory and if they end in .parquet, add them to a list called files
files = [file for file in os.listdir(path) if file.endswith('.parquet')]

datasets = []
for file in files:
    df = pd.read_parquet(path + '/' + file)
    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']))
    gdf.crs = 'EPSG:4326'
    datasets.append(gdf)

# Now datasets list contains GeoDataFrames

In [94]:
# doing some basic stuff with each dataframe
# for range of number of lists in datasets print the datasets number

for d in range(len(datasets)):
    print ("starting " +files[d]) 
    datasets[d] = datasets[d].replace([np.inf, -np.inf], None)
    datasets[d] = datasets[d].replace([np.nan], None)
    datasets[d] = merge_time_columns(datasets[d])
    # appending destination type to travel time and time delta columns
    datasets[d] = datasets[d].rename(columns={col: col + files[d][14:-8] for col in datasets[d].columns if col.startswith('travel_time')})
    datasets[d] = datasets[d].rename(columns={col: col + files[d][14:-8] for col in datasets[d].columns if col.startswith('time_delta')})
    
    datasets[d]['row_col'] = datasets[d]['row_col'].astype(str)
    print ("finished with " +files[d]) 

starting travel_time_to_secondary_schools_fixed.parquet
finished with travel_time_to_secondary_schools_fixed.parquet
starting travel_time_to_semi_dense_urban_optimal.parquet
finished with travel_time_to_semi_dense_urban_optimal.parquet
starting travel_time_to_major_hospitals_optimal.parquet
finished with travel_time_to_major_hospitals_optimal.parquet
starting travel_time_to_health_posts_optimal.parquet
finished with travel_time_to_health_posts_optimal.parquet
starting travel_time_to_health_centers_optimal.parquet
finished with travel_time_to_health_centers_optimal.parquet
starting travel_time_to_all_education_facilities_fixed.parquet
finished with travel_time_to_all_education_facilities_fixed.parquet
starting travel_time_to_primary_schools_fixed.parquet
finished with travel_time_to_primary_schools_fixed.parquet
starting travel_time_to_all_health_facilities_optimal.parquet
finished with travel_time_to_all_health_facilities_optimal.parquet


In [95]:
# merging all dataframes into one
# each dataframe brings it's unique travel times and travel deltas
for d in datasets:
    # I make the first df be the main one and then merge all of the rest into it.
    travel_time_merged = datasets[0]
    for d in range(1, len(datasets)):
        # make a list of values from columns that we're going to merge into the dataframe
        merge_col = []
        for c in list(datasets[d].columns):
            if c.startswith('travel_time') or c.startswith('time_delta'):
                merge_col.append(c)
        # merge the travel_time_merged df with the datasets[d] df on the row_col column
        travel_time_merged = pd.merge(travel_time_merged, datasets[d][['row_col'] + merge_col], on='row_col', how='left')
        # replace NaN values in travel_time_merged with values from datasets[d] wherever they exist
        travel_time_merged = datasets[d][['row_col'] + merge_col].set_index('row_col').combine_first(travel_time_merged.set_index('row_col')).reset_index()
        # fill NA values with values from datasets[d]
        travel_time_merged = travel_time_merged.fillna(datasets[d])


# convert travel_time_merged to geodataframe
travel_time_merged = gpd.GeoDataFrame(travel_time_merged, geometry='geometry')
travel_time_merged.crs = "EPSG:4326"

# drop x_ras_y_ras column
travel_time_merged = travel_time_merged.drop(['x_ras_y_ras'], axis=1)

In [96]:
travel_time_merged_proof = travel_time_merged.copy()

In [97]:
# change travel_time_merged['x_y'] to a string separated by a _
travel_time_merged['x_y'] = travel_time_merged['x_y'].astype(str).str.replace('[', '').str.replace(']', '').str.replace(' ', '').str.replace(',', '_')
# change row_col to a string separated by a _
travel_time_merged['row_col'] = travel_time_merged['row_col'].astype(str).str.replace('[', '').str.replace(']', '').str.replace(' ', '').str.replace(',', '_')

In [98]:
print(travel_time_merged['x_y'][0])
print(travel_time_merged['row_col'][0])

30.4384884_-1.0540862
10_1897


In [99]:
# make rwi equal to rwi times the population value in it's row
travel_time_merged['rwi'] = travel_time_merged['rwi'] * travel_time_merged['population']
travel_time_merged['underweight'] = travel_time_merged['underweight'] * travel_time_merged['population']
# travel_time_merged['male_educational_attainment_mean'] equals the sum the value of male_educational_attainment_mean times the sum of 'males_15_49'+'males_50_64'+'males_65_plus'
travel_time_merged['male_educational_attainment_mean'] = travel_time_merged['male_educational_attainment_mean'] * (travel_time_merged['males_15_49']+travel_time_merged['males_50_64']+travel_time_merged['males_65_plus'])
travel_time_merged['female_educational_attainment_mean'] = travel_time_merged['female_educational_attainment_mean'] * (travel_time_merged['females_15_49']+travel_time_merged['females_50_64']+travel_time_merged['females_65_plus'])


In [100]:
def aggregate_lists(x):
    return "&".join(x)
    
def male_educational_attainment_weighted_average(x):
    # Calculate the weighted average using np.average and weights based on sum of males
    total_males = travel_time_merged.loc[x.index, ['males_15_49', 'males_50_64', 'males_65_plus']].sum(axis=1)
    total_weight = total_males.sum()
    if total_weight == 0:
        # If the total weight is zero, return a default value (e.g., 0) or handle it as per your requirement.
        return 0
    non_zero_mask = (total_males > 0) & (~x.isnull()) # Mask to avoid dividing by zero
    if non_zero_mask.any():
        weighted_avg = np.average(x[non_zero_mask], weights=total_males[non_zero_mask])
        return weighted_avg
    else:
        weighted_avg = np.nan
        return weighted_avg
def female_educational_attainment_weighted_average(x):
    # Calculate the weighted average using np.average and weights based on sum of females
    total_females = travel_time_merged.loc[x.index, ['females_15_49', 'females_50_64', 'females_65_plus']].sum(axis=1)
    total_weight = total_females.sum()
    
    if total_weight == 0:
        # If the total weight is zero, return a default value (e.g., 0) or handle it as per your requirement.
        return 0
    
    non_zero_mask = (total_females > 0) & (~x.isnull())  # Mask to avoid dividing by zero
    if non_zero_mask.any():
        weighted_avg = np.average(x[non_zero_mask], weights=total_females[non_zero_mask])
        return weighted_avg
    else:
        weighted_avg = np.nan
        return weighted_avg

def weighted_average_function(values, weights):
    if weights.sum() == 0:
        return np.nan
    if weights.sum() == np.nan:
        return np.nan
    if weights.sum() == np.inf:
        return np.nan
    else:
        return np.average(values, weights=weights)


joining_rules = {
'row_col': aggregate_lists,
'x_y': aggregate_lists,
'female_educational_attainment_mean': 'sum', 
'females_0_4': 'sum', 
'females_0_9': 'sum', 
'females_10_14': 'sum', 
'females_15_49': 'sum', 
'females_50_64': 'sum', 
'females_5_9': 'sum', 
'females_65_plus': 'sum', 
'male_educational_attainment_mean': 'sum',
'males_0_4': 'sum', 
'males_0_9': 'sum', 
'males_10_14': 'sum', 
'males_15_49': 'sum', 
'males_50_64': 'sum', 
'males_5_9': 'sum', 
'males_65_plus': 'sum', 
'pop_0_4': 'sum', 
'pop_0_9': 'sum', 
'pop_10_14': 'sum', 
'pop_15_49': 'sum', 
'pop_50_64': 'sum', 
'pop_5_9': 'sum', 
'pop_65_plus': 'sum', 
'population': 'sum', 
'pregnancies': 'sum',
'births': 'sum', 
'rwi': 'sum',
'underweight': 'sum', 
'time_delta_constructed_sites_all_education_facilities_fixed': 'mean', 
'time_delta_constructed_sites_all_health_facilities_optimal': 'mean', 
'time_delta_constructed_sites_health_centers_optimal': 'mean', 
'time_delta_constructed_sites_health_posts_optimal': 'mean', 
'time_delta_constructed_sites_major_hospitals_optimal': 'mean', 
'time_delta_constructed_sites_primary_schools_fixed': 'mean', 
'time_delta_constructed_sites_secondary_schools_fixed': 'mean', 
'time_delta_constructed_sites_semi_dense_urban_optimal': 'mean', 
'time_delta_no_sites_all_education_facilities_fixed': 'mean', 
'time_delta_no_sites_all_health_facilities_optimal': 'mean', 
'time_delta_no_sites_health_centers_optimal': 'mean', 
'time_delta_no_sites_health_posts_optimal': 'mean', 
'time_delta_no_sites_major_hospitals_optimal': 'mean', 
'time_delta_no_sites_primary_schools_fixed': 'mean', 
'time_delta_no_sites_secondary_schools_fixed': 'mean', 
'time_delta_no_sites_semi_dense_urban_optimal': 'mean', 
'travel_time_all_education_facilities_fixed': 'mean', 
'travel_time_all_health_facilities_optimal': 'mean', 
'travel_time_constructed_sites_all_education_facilities_fixed': 'mean', 
'travel_time_constructed_sites_all_health_facilities_optimal': 'mean', 
'travel_time_constructed_sites_health_centers_optimal': 'mean', 
'travel_time_constructed_sites_health_posts_optimal': 'mean', 
'travel_time_constructed_sites_major_hospitals_optimal': 'mean', 
'travel_time_constructed_sites_primary_schools_fixed': 'mean', 
'travel_time_constructed_sites_secondary_schools_fixed': 'mean', 
'travel_time_constructed_sites_semi_dense_urban_optimal': 'mean', 
'travel_time_health_centers_optimal': 'mean', 
'travel_time_health_posts_optimal': 'mean', 
'travel_time_major_hospitals_optimal': 'mean', 
'travel_time_no_sites_all_education_facilities_fixed': 'mean', 
'travel_time_no_sites_all_health_facilities_optimal': 'mean', 
'travel_time_no_sites_health_centers_optimal': 'mean', 
'travel_time_no_sites_health_posts_optimal': 'mean', 
'travel_time_no_sites_major_hospitals_optimal': 'mean', 
'travel_time_no_sites_primary_schools_fixed': 'mean', 
'travel_time_no_sites_secondary_schools_fixed': 'mean', 
'travel_time_no_sites_semi_dense_urban_optimal': 'mean', 
'travel_time_primary_schools_fixed': 'mean', 
'travel_time_secondary_schools_fixed': 'mean', 
'travel_time_semi_dense_urban_optimal': 'mean',
}

In [101]:
def make_hexagons(resolution, write_to_file):
    # make travel_time_merged_hex a geopandas dataframe that matches travel_time_merged
    travel_time_merged_hex = travel_time_merged.copy()
    travel_time_merged_hex = gpd.GeoDataFrame(travel_time_merged_hex, geometry='geometry')
    travel_time_merged_hex.crs = "EPSG:4326"

    travel_time_merged_hex['h3-index'] = None

    for idx, row in travel_time_merged_hex.iterrows():
        try:
            lat = row['geometry'].y
            lon = row['geometry'].x
            travel_time_merged_hex.at[idx, 'h3-index'] = h3.geo_to_h3(lat, lon, resolution)
        except:
            pass

    # merge all rows that match on h3-index following joining rules
    travel_time_merged_hex = travel_time_merged_hex.groupby('h3-index').agg(joining_rules)
    travel_time_merged_hex = travel_time_merged_hex.reset_index()
    # drop any rows without a h3-index
    travel_time_merged_hex = travel_time_merged_hex.dropna(subset=['h3-index'])
    # convert h3-index to polygon
    travel_time_merged_hex['geometry'] = travel_time_merged_hex['h3-index'].apply(lambda x: h3.h3_to_geo_boundary(x, True))
    # make geometry column a polygon
    travel_time_merged_hex['geometry'] = travel_time_merged_hex['geometry'].apply(lambda x: Polygon(x))
    # convert travel_time_merged_hex to geodataframe
    travel_time_merged_hex = gpd.GeoDataFrame(travel_time_merged_hex, geometry='geometry')
    travel_time_merged_hex.crs = "EPSG:4326"
    if write_to_file == True:
        travel_time_merged_hex.to_file("./synced-data/rwa_travel_time_hex-"+str(resolution)+".geojson", driver='GeoJSON', na='null')
    return travel_time_merged_hex

In [102]:
hex8 = make_hexagons(8, False)

In [52]:
# ToDelete
def normalize_by_population(df):
    try:
        df['rwi'] = df['rwi'] / hex8['population']
    except:
        df['rwi'] = np.nan
    try:
        df['underweight'] = df['underweight'] / hex8['population']
    except:
        df['underweight'] = np.nan
    try:
        df['male_educational_attainment_mean'] = df['male_educational_attainment_mean'] / (df['males_15_49']+df['males_50_64']+df['males_65_plus'])
    except:
        df['male_educational_attainment_mean'] = np.nan
    try:
        df['female_educational_attainment_mean'] = df['female_educational_attainment_mean'] / (df['females_15_49']+df['females_50_64']+df['females_65_plus'])
    except:
        df['female_educational_attainment_mean'] = np.nan
    return df
hex8 = normalize_by_population()

In [57]:
# print type of rwi column of hex8
print(type(hex8['population'][0])) 

<class 'float'>


In [104]:
# Normalizing the weighted population columns
hex8['rwi'] = hex8.apply(lambda row: row['rwi'] / row['population'] if row['population'] != 0 else np.nan, axis=1)
hex8['underweight'] = hex8.apply(lambda row: row['underweight'] / row['population'] if row['population'] != 0 else np.nan, axis=1)
hex8['male_educational_attainment_mean'] = hex8.apply(lambda row: row['male_educational_attainment_mean'] / (row['males_15_49'] + row['males_50_64'] + row['males_65_plus']) if (row['males_15_49'] + row['males_50_64'] + row['males_65_plus']) != 0 else np.nan, axis=1)
hex8['female_educational_attainment_mean'] = hex8.apply(lambda row: row['female_educational_attainment_mean'] / (row['females_15_49'] + row['females_50_64'] + row['females_65_plus']) if (row['females_15_49'] + row['females_50_64'] + row['females_65_plus']) != 0 else np.nan, axis=1)

In [107]:
# Checking to see normalizaton worked
print(hex8['female_educational_attainment_mean'].max())
print(hex8['male_educational_attainment_mean'].max())

8.658925975996027
8.966146346597323


In [108]:
# should do this after merging with hexagons so  that we don't lose precision
# change datatypes of columns
print(hex8.dtypes)
int_list = ['time_delta_constructed_sites_all_education_facilities_fixed', 'time_delta_constructed_sites_all_health_facilities_optimal', 'time_delta_constructed_sites_health_centers_optimal', 'time_delta_constructed_sites_health_posts_optimal', 'time_delta_constructed_sites_major_hospitals_optimal', 'time_delta_constructed_sites_primary_schools_fixed', 'time_delta_constructed_sites_secondary_schools_fixed', 'time_delta_constructed_sites_semi_dense_urban_optimal', 'time_delta_no_sites_all_education_facilities_fixed', 'time_delta_no_sites_all_health_facilities_optimal', 'time_delta_no_sites_health_centers_optimal', 'time_delta_no_sites_health_posts_optimal', 'time_delta_no_sites_major_hospitals_optimal', 'time_delta_no_sites_primary_schools_fixed', 'time_delta_no_sites_secondary_schools_fixed', 'time_delta_no_sites_semi_dense_urban_optimal', 'travel_time_all_education_facilities_fixed', 'travel_time_all_health_facilities_optimal', 'travel_time_constructed_sites_all_education_facilities_fixed', 'travel_time_constructed_sites_all_health_facilities_optimal', 'travel_time_constructed_sites_health_centers_optimal', 'travel_time_constructed_sites_health_posts_optimal', 'travel_time_constructed_sites_major_hospitals_optimal', 'travel_time_constructed_sites_primary_schools_fixed', 'travel_time_constructed_sites_secondary_schools_fixed', 'travel_time_constructed_sites_semi_dense_urban_optimal', 'travel_time_health_centers_optimal', 'travel_time_health_posts_optimal', 'travel_time_major_hospitals_optimal', 'travel_time_no_sites_all_education_facilities_fixed', 'travel_time_no_sites_all_health_facilities_optimal', 'travel_time_no_sites_health_centers_optimal', 'travel_time_no_sites_health_posts_optimal', 'travel_time_no_sites_major_hospitals_optimal', 'travel_time_no_sites_primary_schools_fixed', 'travel_time_no_sites_secondary_schools_fixed', 'travel_time_no_sites_semi_dense_urban_optimal', 'travel_time_primary_schools_fixed', 'travel_time_secondary_schools_fixed', 'travel_time_semi_dense_urban_optimal', 'males_0_4', 'males_0_9', 'males_10_14', 'males_15_49', 'males_50_64', 'males_5_9', 'males_65_plus', 'pop_0_4', 'pop_0_9', 'pop_10_14', 'pop_15_49', 'pop_50_64', 'pop_5_9', 'pop_65_plus', 'population', 'females_0_4', 'females_0_9', 'females_10_14', 'females_15_49', 'females_50_64', 'females_5_9', 'females_65_plus', 'births', 'pregnancies', ]
float_list = ['rwi', 'underweight', 'female_educational_attainment_mean', 'male_educational_attainment_mean']
string_list = ['x_y', 'row_col']
for col in int_list:
        hex8[col] = hex8['travel_time_semi_dense_urban_optimal'].astype(int)
for col in float_list:
    hex8[col] = pd.to_numeric(hex8[col], errors='coerce', downcast='float')
    hex8[col] = hex8[col].round(4)

print(hex8.dtypes)

h3-index                                           object
row_col                                            object
x_y                                                object
female_educational_attainment_mean                float64
females_0_4                                        object
                                                   ...   
travel_time_no_sites_semi_dense_urban_optimal      object
travel_time_primary_schools_fixed                  object
travel_time_secondary_schools_fixed                object
travel_time_semi_dense_urban_optimal               object
geometry                                         geometry
Length: 72, dtype: object
h3-index                                           object
row_col                                            object
x_y                                                object
female_educational_attainment_mean                float32
females_0_4                                         int64
                                              

In [109]:
# drop x_y from hex8. We won't use this in the app
hex8 = hex8.drop(['x_y'], axis=1)

In [110]:
hex8.to_file("./synced-data/rwa_travel_time_hex-8.geojson", driver='GeoJSON', na='null')


# Creating a hex to subregion lookup table

In [112]:
# create a dataframe with just h3-index and row_col
lookup = hex8[['h3-index', 'row_col']]

In [114]:
# create a new dataframe called lookup_full that has two columns: h3-index and row_col. row_col is the values from each row split on & with the h3-index repeated for each row_col value
lookup_full = pd.DataFrame(columns=['h3-index', 'row_col'])
for idx, row in lookup.iterrows():
    row_col_list = row['row_col'].split('&')
    for row_col in row_col_list:
        lookup_full = lookup_full.append({'h3-index': row['h3-index'], 'row_col': row_col}, ignore_index=True)


AttributeError: 'DataFrame' object has no attribute 'append'

In [115]:

# Assuming you have a DataFrame called 'lookup' with columns 'h3-index' and 'row_col'

# create a list to store dictionaries for each row in the new DataFrame
data = []

for idx, row in lookup.iterrows():
    row_col_list = row['row_col'].split('&')
    for row_col in row_col_list:
        data.append({'h3-index': row['h3-index'], 'row_col': row_col})

# create the new DataFrame 'lookup_full' using the list of dictionaries
lookup_full = pd.DataFrame(data)

# If needed, you can set the data types of the columns explicitly, for example:
# lookup_full = pd.DataFrame(data, dtype={'h3-index': int, 'row_col': str})


In [116]:
lookup_full

,h3-index,row_col
0,886ad80001fffff,1334_1576
1,886ad80001fffff,1335_1573
2,886ad80001fffff,1335_1574
3,886ad80001fffff,1335_1575
4,886ad80001fffff,1335_1576
...,...,...
1150599,886adeb76dfffff,760_600
1150600,886adeb76dfffff,760_601
1150601,886adeb76dfffff,760_602
1150602,886adeb893fffff,974_516


In [118]:
# how many unique values of h3-index are there
len(lookup_full['h3-index'].unique())

26138

In [124]:
# write lookup_full to json file
lookup_full.to_json('./synced-data/subregion-to-hex-index.json', orient='records')

In [126]:
# write lookup_full to csv file
lookup_full.to_csv('./synced-data/subregion-to-hex-index.csv', index=False)